In [1]:
import numpy as np
import tensorflow as tf

In [2]:
model = tf.keras.models.load_model('elu_adam.h5')

In [3]:
test_input = np.array([0.589869,-0.807209,0.0216254,0.374714,-0.258722,0.89031,-0.0590713,-0.901398,-0.428943,-0.582088,-0.812846,-0.0213401,-0.466998,-0.258784,0.845543,0.229916,-0.669814,0.706037,-0.0207609,0.999555,0.0214356,-0.0207609,0.999555,0.0214356,0.932139,-0.350875,-0.0894676,0.432187,-0.884066,0.177881,0.307225,-0.622432,0.719855,-0.932139,-0.350874,-0.0894673,-0.366027,-0.81478,0.44962,-0.243548,-0.810353,0.532928
                      ]);
test_input = test_input[np.newaxis,...]
test_input.shape

(1, 42)

In [4]:
result = model.predict(test_input)

In [5]:
result

array([[-0.00555471, -0.06832403, -0.07481331,  0.03448331,  0.02256336,
        -0.2855381 , -0.08109752,  0.12458856,  0.1092386 , -0.02902506,
        -0.10049238, -0.54094243,  0.13261864, -0.1598331 , -0.31030914,
        -0.01860901, -0.07351683, -0.08230709, -0.01709482, -0.08086209,
        -0.05874683, -0.31309584, -0.1021783 ,  0.13576868, -0.33643436,
        -0.19155769, -0.10506376,  0.32434642,  0.00320716,  0.14566986,
         0.30687314,  0.11111112,  0.21967268]], dtype=float32)

In [6]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 42)]              0         
_________________________________________________________________
dense (Dense)                (None, 100)               4300      
_________________________________________________________________
dense_1 (Dense)              (None, 150)               15150     
_________________________________________________________________
dense_2 (Dense)              (None, 50)                7550      
_________________________________________________________________
dense_3 (Dense)              (None, 33)                1683      
Total params: 28,683
Trainable params: 28,683
Non-trainable params: 0
_________________________________________________________________


## 重新将模型保存为pb格式
https://stackoverflow.com/questions/45466020/how-to-export-keras-h5-to-tensorflow-pb
https://github.com/leimao/Frozen_Graph_TensorFlow/blob/master/TensorFlow_v2/train.py

In [7]:
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

In [8]:
# Convert Keras model to ConcreteFunction
full_model = tf.function(lambda x: model(x))
full_model = full_model.get_concrete_function(
    tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))

# Get frozen ConcreteFunction
frozen_func = convert_variables_to_constants_v2(full_model)
frozen_func.graph.as_graph_def()

layers = [op.name for op in frozen_func.graph.get_operations()]
print("-" * 50)
print("Frozen model layers: ")
for layer in layers:
    print(layer)

print("-" * 50)
print("Frozen model inputs: ")
print(frozen_func.inputs)
print("Frozen model outputs: ")
print(frozen_func.outputs)

# Save frozen graph from frozen ConcreteFunction to hard drive
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir="./frozen_models",
                  name="frozen_graph.pb",
                  as_text=False)

--------------------------------------------------
Frozen model layers: 
x
model/dense/MatMul/ReadVariableOp/resource
model/dense/MatMul/ReadVariableOp
model/dense/MatMul
model/dense/BiasAdd/ReadVariableOp/resource
model/dense/BiasAdd/ReadVariableOp
model/dense/BiasAdd
model/dense/Tanh
model/dense_1/MatMul/ReadVariableOp/resource
model/dense_1/MatMul/ReadVariableOp
model/dense_1/MatMul
model/dense_1/BiasAdd/ReadVariableOp/resource
model/dense_1/BiasAdd/ReadVariableOp
model/dense_1/BiasAdd
model/dense_1/Tanh
model/dense_2/MatMul/ReadVariableOp/resource
model/dense_2/MatMul/ReadVariableOp
model/dense_2/MatMul
model/dense_2/BiasAdd/ReadVariableOp/resource
model/dense_2/BiasAdd/ReadVariableOp
model/dense_2/BiasAdd
model/dense_2/Tanh
model/dense_3/MatMul/ReadVariableOp/resource
model/dense_3/MatMul/ReadVariableOp
model/dense_3/MatMul
model/dense_3/BiasAdd/ReadVariableOp/resource
model/dense_3/BiasAdd/ReadVariableOp
model/dense_3/BiasAdd
model/dense_3/Tanh
Identity
--------------------------

'./frozen_models/frozen_graph.pb'